In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
import cv2
import random
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import numpy as np
import time
import argparse
import shutil
import string
import pandas as pd
import math as m
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import glob
import os


In [ ]:
def getCornersFromH4pt(corner1, H4pt):
    corners1 = np.array(corner1.copy())
    del_corners = H4pt.reshape(2,4).T
    corners2 = corners1 + del_corners
    return corners2

def drawCorners(image, corners, color):

    corners_ = np.array(corners.copy())
    r = corners_[2,:].copy()
    corners_[2,:] = corners_[3,:]
    corners_[3,:] = r
    corners_ = corners_.reshape(-1,1,2)
#     print(corners_)
    corners_ = corners_.astype(int)
    image_corners = cv2.polylines(image.copy(),[corners_],True,color, 4)
    return image_corners

def getHfromH4pt(corners1, H4pt):
#     print("H4pt is: ")
#     print(H4pt.reshape(2,4).T)

    del_corners = H4pt.reshape(2,4).T
    
    corners1 = np.array(corners1)
#     print("corner1 is: ")
#     print(corners1)

    corners2 = corners1 + del_corners
#     print("corner2 is: ")
#     print(corners2)

    H = cv2.getPerspectiveTransform(np.float32(corners1), np.float32(corners2))
#     print("H is:")
#     print(H)
    return H

def warpImage(img, corners, H):
    image = img.copy()
    h, w, _= image.shape

    corners_ = np.array(corners)
    corners_ = corners_.reshape((-1,1,2))

    image_transformed = cv2.warpPerspective(image, H, (w,h))
    corner_transformed = cv2.perspectiveTransform(np.float32(corners_), H)
    corner_transformed = corner_transformed.astype(int)
    
    return image_transformed, corner_transformed

In [ ]:
def Visualise_supervised(i, BasePath, SavePath, train_labels, pointsList, image):
    
    Y_Pred = np.load(SavePath+'/h4pt_Pred.npy')
    print(Y_Pred.shape)


    corners_a = pointsList[i,:,:,0]
    corners_b = pointsList[i,:,:,1]

    imA = cv2.imread(image)

    H_AB = getHfromH4pt(corners_a, train_labels[i])
    imB, corners_b_cal = warpImage(imA, corners_a, H_AB)

    imA_corners = drawCorners(imA, corners_a, (0,0,255))
    imB_corners = drawCorners(imB, corners_b_cal, (0,0,255))

    mae = mean_absolute_error(Y_Pred[i], train_labels[i])
    print("Mean absolute Error for image at index ",i, ":  ",mae)

    corners_b_pred = getCornersFromH4pt(corners_a, Y_Pred[i])
    # imA_corners = drawCorners(imA, pts1, (0,0,255))
    imB_corners_pred = drawCorners(imB_corners, corners_b_pred, (0,255,255))
    imB_corners_pred = cv2.putText(imB_corners_pred, "MCE: "+str(round(mae,3)),(150,230),cv2.FONT_HERSHEY_SIMPLEX,0.75,(255,0,0),2,cv2.LINE_AA)
    return np.hstack((imA_corners,imB_corners_pred))

In [ ]:
def homography_model(img, image_size, batch_size):
  x = tf.layers.conv2d(inputs=img, name='Conv2D1', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm1')
  x = tf.nn.relu(x, name='Relu1')

  x = tf.layers.conv2d(inputs=x, name='Conv2D2', padding='same',filters=64, kernel_size=[3,3], activation=None) 
  x = tf.layers.batch_normalization(x, name='BatchNorm2')
  x = tf.nn.relu(x, name='Relu2')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D3', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm3')
  x = tf.nn.relu(x, name='Relu3')

  x = tf.layers.conv2d(inputs=x, name='Conv2D4', padding='same',filters=64, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm4')
  x = tf.nn.relu(x, name='Relu4')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D5', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm5')
  x = tf.nn.relu(x, name='Relu5')

  x = tf.layers.conv2d(inputs=x, name='Conv2D6', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm6')
  x = tf.nn.relu(x, name='Relu6')

  x = tf.layers.max_pooling2d(inputs=x, pool_size=[2,2], strides=2)

  x = tf.layers.conv2d(inputs=x, name='Conv2D7', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm7')
  x = tf.nn.relu(x, name='Relu7')

  x = tf.layers.conv2d(inputs=x, name='Conv2D8', padding='same',filters=128, kernel_size=[3,3], activation=None)
  x = tf.layers.batch_normalization(x, name='BatchNorm8')
  x = tf.nn.relu(x, name='Relu8')

  #flattening layer
  x = tf.contrib.layers.flatten(x)

  #Fully-connected layers
  x = tf.layers.dense(inputs=x, name='FC1',units=1024, activation=tf.nn.relu)
  x = tf.layers.dropout(x,rate=0.5, training=True,name='Dropout')
  x = tf.layers.batch_normalization(x, name='BatchNorm9')
  H4 = tf.layers.dense(inputs=x, name='FCfinal',units=8, activation=None)

  return H4

In [ ]:
def GenerateBatch(test_files_pa, test_files_pb, TrainLabels, MiniBatchSize):

  patches_ab = []
  labels = []

  for i in range(MiniBatchSize):
    index = random.randint(0, len(test_files_pa)-1) 
    patch_a = cv2.imread(test_files_pa[i], cv2.IMREAD_GRAYSCALE)
    patch_b = cv2.imread(test_files_pb[i], cv2.IMREAD_GRAYSCALE)

    patch_a = np.float32(patch_a)
    patch_b = np.float32(patch_b)  

    #combine images along depth
    patch_ab = np.dstack((patch_a, patch_b))     
    labels.append(TrainLabels[i,:].reshape(8,))
    patches_ab.append(patch_ab)

  return np.array(patches_ab), np.array(labels)

def Test_sup(ImgPH,ModelPath, test_files_PA, test_files_PB, train_labels, image_files, SavePath):

    if(not (os.path.isdir(SavePath))):
        print(SavePath, "  was not present, creating the folder...")
        os.makedirs(SavePath)
    imgData,labels = GenerateBatch(test_files_PA, test_files_PB, train_labels, len(test_files_PA))

    H4pt = homography_model(ImgPH, [128,128,2], len(test_files_PA))
    Saver = tf.train.Saver()
  
    with tf.Session() as sess:
        Saver.restore(sess, ModelPath)
        print('Number of parameters in this model are %d ' % np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))
        
    
        imgData=np.array(imgData).reshape(len(test_files_PA),128,128,2)
        
        FeedDict = {ImgPH: imgData}
        Predicted = sess.run(H4pt,FeedDict)
        h4pt_pred = np.array(Predicted)
        print(h4pt_pred)
        print(h4pt_pred.shape)
        
    np.save(SavePath+"/h4pt_Pred.npy",h4pt_pred)

In [ ]:
def main():
    tf.reset_default_graph()
    # base_dir = os.getcwd()
    base_path ='/content/drive/MyDrive/Project_1' 
    test_files_PA = glob.glob( base_path + '/S_val/P_A'+ '/*.jpg')
    test_files_PB = glob.glob( base_path + '/S_val/P_B'+ '/*.jpg')
    image_files = glob.glob( base_path + '/S_val/I_A'+ '/*.jpg')
    NumTestSamples = len(test_files_PA)
    
    ModelPath='/content/drive/MyDrive/Project_1/supervised_checkpoints/19model.ckpt'
    SavePath = '/content/drive/MyDrive/Project_1/supervised_test'
    train_labels = pd.read_csv(base_path + '/S_val/H_4pt.csv', index_col = False)
    train_labels = train_labels.to_numpy()
    ImgPH=tf.placeholder(tf.float32, shape=(len(test_files_PA), 128, 128, 2))
    Test_sup(ImgPH,ModelPath, test_files_PA, test_files_PB, train_labels, image_files, SavePath)
    pointsList = np.load(base_path + '/S_val/points_list.npy')
    rand_i = np.random.randint(0,len(test_files_PA)-1, size=5)
    for i in rand_i:
        image = image_files[i]
        comparison = Visualise_supervised(i, base_path + '/S_val', SavePath, train_labels, pointsList, image)
        cv2.imwrite(SavePath+'/comparison'+ str(i)+'.png',comparison)
    
    print('Check Results/supervised folder..')     
if __name__ == '__main__':
    main()
 

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/Project_1/supervised_checkpoints/19model.ckpt
Number of parameters in this model are 34195848 
[[-9.010777  -7.0060797 10.01232   ... -4.0031695  1.0021203 15.026427 ]
 [-9.0103855 -7.0056806 10.011926  ... -4.0033216  1.002371  15.026286 ]
 [-9.011533  -7.0067954 10.013615  ... -4.003926   1.0022521 15.029782 ]
 ...
 [-9.008837  -7.005301  10.01055   ... -4.0029945  1.0022489 15.021788 ]
 [-9.010071  -7.005786  10.011204  ... -4.0032506  1.0022935 15.025472 ]
 [-9.008175  -7.0052524 10.009947  ... -4.0031157  1.0022079 15.022487 ]]
(1000, 8)
(1000, 8)
Mean absolute Error for image at index  113 :   13.764420886640437
(1000, 8)
Mean absolute Error for image at index  394 :   20.26564726990182
(1000, 8)
Mean absolute Error for image at index  681 :   21.508347322349437
(1000, 8)
Mean absolute Error for image at index  509 :   19.014017799752764
(1000, 8)
Mean absolute Error for image at index  639 :   18.263399141957052
Ch